In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot
import math
%matplotlib inline 


In [2]:
df = pd.read_csv('dataset.csv')
df = df.drop(columns = 'Index')
df

,Energy,Correlation,Dissimilarity,Homogenity,Contrast,class_label
0,0.019349,0.867882,18.958477,0.099115,1128.419845,0
1,0.018590,0.895155,16.439161,0.099919,734.834338,0
2,0.020157,0.904695,16.137180,0.115214,782.162402,0
3,0.112826,0.943806,12.886319,0.287135,841.671875,0
4,0.112826,0.943806,12.886319,0.287135,841.671875,0
5,0.119517,0.942602,13.027313,0.297123,853.388410,0
6,0.119517,0.942602,13.027313,0.297123,853.388410,0
7,0.124367,0.940236,13.142286,0.303815,879.873708,0
8,0.122299,0.937820,13.223241,0.294432,866.365342,0
9,0.043661,0.941740,10.756582,0.218150,523.159879,0


In [3]:
X = df.drop(columns = 'class_label')
y = df['class_label']

In [4]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
clf = DecisionTreeClassifier(max_depth=3, random_state=1234)
model = clf.fit(X, y)

In [5]:
from sklearn.metrics import accuracy_score
predictions = clf.predict(X_test)

In [6]:
#defining various steps required for the genetic algorithm
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)
        chromosome[:int(0.3*n_feat)]=False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population):
    scores = []
    for chromosome in population:
        clf.fit(X_train.iloc[:,chromosome],y_train)
        predictions = clf.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen

def crossover(pop_after_sel):
    population_nextgen=pop_after_sel
    for i in range(len(pop_after_sel)):
        child=pop_after_sel[i]
        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen

def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

def generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, y_train, y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print(scores[:2])
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [8]:
chromo,score=generations(size=31,n_feat=5,n_parents=3,mutation_rate=0.10,
                     n_gen=10,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)
clf.fit(X_train.iloc[:,chromo[-1]],y_train)
predictions = clf.predict(X_test.iloc[:,chromo[-1]])
print("Accuracy score after genetic algorithm is= "+str(accuracy_score(y_test,predictions)))

<ipython-input-6-6b4f1b4716f3>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  chromosome = np.ones(n_feat,dtype=np.bool)


[1.0, 1.0]
[0.9, 0.9]
[0.8, 0.8]
[1.0, 1.0]
[1.0, 1.0]
[1.0, 1.0]
[1.0, 1.0]
[1.0, 1.0]
[1.0, 1.0]
[1.0, 1.0]
Accuracy score after genetic algorithm is= 1.0


In [9]:
chromo[-1]

array([False,  True,  True,  True,  True])